# Modelos y Simulación 2024

## Trabajo Práctico Final:
## Hospital

### Alumnos:
- Ana Enzo
- Barea Matias
- Irigoyen Carlos
- Morillo Alen

### Profesores: Lic. Claudia López de Munain - Lic. Bruno Alfredo Pazos

In [ ]:
from hospital import *

In [16]:
''' TEST SIMULACION'''
import numpy as np
import matplotlib.pyplot as plt
import random

# Tiempo simulacion
ANIOS_SIMULACION = 2
DIAS_SIMULACION = 365 * ANIOS_SIMULACION # 730 dias

# Valores promedios
PROMEDIO_KITS_MENSUALES = 5000 # sigue una distribucion de poisson
MEDIA_PACIENTES_DIARIOS = 110 # sigue una distribucion de poisson
PROMEDIO_TIEMPO_CIRUGIA = 1.5 # tiempo en horas
PROB_RESERVA_QUIRONAFO = 0.43 # 43% Uniforme
PROMEDIO_TIEMPO_INTERNACION = 2  # dias, sigue una distribucion exponencial
PROMEDIO_CIRUGIAS_DIARIAS = 8 # por quirofano

# Parametros iniciales de la simulacion
KITS_INICIALES = np.random.poisson(PROMEDIO_KITS_MENSUALES)
CAMAS_TOTALES = 310 # parametro de entrada para comprarar con los objetivos propuestos
CANT_QUIROFANOS = 4 # parametro de entrada para comparar con los objetivos propuestos
CANT_HORAS_ATENCION_QUIROFANO = 24 # tiempo en horas
KITS_REPOSICION_DIARIA = 100 # parametro de entrada para comprarar con los objetivos propuestos

# ocupacion_quirofano_1 = 0
# ocupacion_quirofano_2 = 0
# ocupacion_quirofano_3 = 0
# ocupacion_quirofano_4 = 0

# Arrays
disponibilidad_camas = [CAMAS_TOTALES] * DIAS_SIMULACION

# Colas
quirofanos = {
    'quirofano 1' : [],
    'quirofano 2' : [],
    'quirofano 3' : [],
    'quirofano 4' : [],
    'quirofano 5' : [],
    'quirofano 6' : [],
    'quirofano 7' : [],
    'quirofano 8' : []
}

# Paciente. Atributos: DIA DE INTERNACION Y DIA DE ALTA
paciente_base = {
    'dia_internacion': 0,
    'dia_alta': 0,
}

# Calcular probabilidades
def generar_demanda_diaria():
    return np.random.poisson(MEDIA_PACIENTES_DIARIOS) # CANTIDAD DE PACIENTES QUE INGRESAN DIARIAMENTE

def generar_dias_internacion():
    return np.random.exponential(PROMEDIO_TIEMPO_INTERNACION) # TIEMPO EN EL QUE LOS PACIENTES ESTAN INTERNADOS OCUPANDO UNA CAMA

def generar_cantidad_cirugias_diarias():
    return np.random.poisson(PROMEDIO_CIRUGIAS_DIARIAS)

def determinar_tiempo_operacion():
    return np.random.exponential(PROMEDIO_TIEMPO_CIRUGIA) # TIEMPO DE SALIDA DE PACIENTE DEL QUIROFANO

def asignar_cama(dia, cant_dias_internacion):
    for dia_estadia in range(dia, dia+cant_dias_internacion):
        disponibilidad_camas[dia_estadia] -= 1

def crear_paciente(dia_internacion, dia_alta):
    paciente = paciente_base.copy()
    paciente['dia_internacion'] = dia_internacion
    paciente['dia_alta'] = dia_alta
    return paciente

# def determinar_quirofano_con_disponibilidad(ocupacion_quirofanos):
#     for key, ocupacion in ocupacion_quirofanos.items():
#         if ocupacion <= CANT_HORAS_ATENCION_QUIROFANO:
#             return key
#     return None

def determinar_quirofano():
    quirofanos_keys = list(quirofanos.keys())
    key = quirofanos_keys[random.randint(0,7)]
    return key


def asignar_quirofano(paciente, quirofano):
    quirofanos[quirofano].append(paciente)

def procesar_pacientes(dia, ocupacion_quirofanos, cantidad_pacientes):
    for i in range(cantidad_pacientes):               
        cant_dias_internacion = generar_dias_internacion()
        cant_dias_internacion = 1 if cant_dias_internacion < 1 else cant_dias_internacion # Tomamos los pacientes < 0 dias internacion como = 1(un) dia de internacion
        dias_int = int(cant_dias_internacion)
        if dia+dias_int <= DIAS_SIMULACION:
            asignar_cama(dia, dias_int)
            necesita_quirofano = np.random.uniform(0,1)
            if necesita_quirofano >= 0.43:
                paciente = crear_paciente(dia, dia+dias_int)
                quirofano = determinar_quirofano()
                asignar_quirofano(paciente, quirofano)
        else:
            break

def hay_kits(kits_disponibles):
    return kits_disponibles > 0

def simulacion():
    
    # Contadores globales totales
    total_ocupacion_quirofanos = []
    tiempo_espera_quirofanos = []
    total_kits_utilizados = []
    cirugias_reprogramadas_por_insumos = 0
    cirugias_reprogramadas_por_tiempo = 0
    cirugias_concretadas = 0
    cirugias_rechazadas = 0
    pacientes_rechazados = 0

    kits_disponibles = KITS_INICIALES
    
    for dia in range(DIAS_SIMULACION):
        
        ocupacion_quirofanos = {
            'quirofano 1' : 0,
            'quirofano 2' : 0,
            'quirofano 3' : 0,
            'quirofano 4' : 0,
            'quirofano 5' : 0,
            'quirofano 6' : 0,
            'quirofano 7' : 0,
            'quirofano 8' : 0
        }

        kits_disponibles += KITS_REPOSICION_DIARIA #Incrementar kits diariamente
        
        '''ORGANIZACION DE LA DEMANDA'''

        llegadas = generar_demanda_diaria()

        if disponibilidad_camas[dia] >= llegadas:
            procesar_pacientes(dia, ocupacion_quirofanos, llegadas)
        else:
            pacientes_rechazados += llegadas - disponibilidad_camas[dia]
            camas_disponibles = disponibilidad_camas[dia]
            procesar_pacientes(dia, ocupacion_quirofanos, camas_disponibles)

        '''INICIO HORARIO ATENCION'''

        # while (((ocupacion_quirofanos['quirofano 1'] <= CANT_HORAS_ATENCION_QUIROFANO)
        #         or (ocupacion_quirofanos['quirofano 2'] <= CANT_HORAS_ATENCION_QUIROFANO)
        #         or (ocupacion_quirofanos['quirofano 3'] <= CANT_HORAS_ATENCION_QUIROFANO)
        #         or (ocupacion_quirofanos['quirofano 4'] <= CANT_HORAS_ATENCION_QUIROFANO))
        #         and hay_kits(kits_disponibles)):
        
        for key in quirofanos:
            # cirugias_diarias = generar_cantidad_cirugias_diarias()
            
            # if (len(quirofanos[key]) > cirugias_diarias):
            #     cirugias_reprogramadas_por_disp += len(quirofanos[key]) - cirugias_diarias

            # for cirugia in range(cirugias_diarias):
            while (ocupacion_quirofanos[key] <= CANT_HORAS_ATENCION_QUIROFANO and len(quirofanos[key]) > 0 and hay_kits(kits_disponibles)):
                paciente = quirofanos[key].pop(0)
                if dia <= paciente['dia_alta']:
                    if hay_kits(kits_disponibles):
                        ocupacion_quirofanos[key] += determinar_tiempo_operacion()
                        kits_disponibles -= 1
                        cirugias_concretadas += 1
                        tiempo_espera_quirofanos.append(dia - paciente['dia_internacion'])
                    else:
                        cirugias_reprogramadas_por_insumos += 1
                        quirofanos[key].insert(0, paciente)
            
                else:
                    pacientes_rechazados += 1
                    cirugias_rechazadas += 1
            if kits_disponibles == 0:
                cirugias_reprogramadas_por_insumos += len(quirofanos[key])
            else:
                cirugias_reprogramadas_por_tiempo += len(quirofanos[key])                    
        total_kits_utilizados.append(kits_disponibles)
        total_ocupacion_quirofanos.append(ocupacion_quirofanos)

    print('tiempo total de ocupacion del quirofano', total_ocupacion_quirofanos)
    print('quirofanos con pacientes', quirofanos['quirofano 1'])
    print('quirofanos con pacientes', quirofanos['quirofano 2'])
    print('quirofanos con pacientes', quirofanos['quirofano 3'])
    print('quirofanos con pacientes', quirofanos['quirofano 4'])
    print('tiempo de espera quirofano', tiempo_espera_quirofanos)
    print('camas disponibles', disponibilidad_camas)
    print('kits diarios disponibles', total_kits_utilizados)
    print('pacientes rechazados', pacientes_rechazados)
    print('cirugias reprogramadas por tiempo atencion quirofano', cirugias_reprogramadas_por_tiempo)
    print('cirugias reprogramadas por insumos', cirugias_reprogramadas_por_insumos)
    print('cirugias concretadas', cirugias_concretadas)
    print('cirugias rechazadas', cirugias_rechazadas)
    print('kits_disponibles', kits_disponibles)

simulacion()

tiempo total de ocupacion del quirofano [{'quirofano 1': 19.312135777013452, 'quirofano 2': 19.3589160506722, 'quirofano 3': 7.217420377477268, 'quirofano 4': 0.8921366809053836, 'quirofano 5': 3.2283652658971906, 'quirofano 6': 4.5322862797904175, 'quirofano 7': 12.395991432138773, 'quirofano 8': 12.185114792949218}, {'quirofano 1': 9.153474444465653, 'quirofano 2': 11.123138422443322, 'quirofano 3': 8.099140238302796, 'quirofano 4': 8.68388581186634, 'quirofano 5': 14.0375775629288, 'quirofano 6': 10.59011137574883, 'quirofano 7': 11.393373002667666, 'quirofano 8': 16.798678106171515}, {'quirofano 1': 8.797110752595334, 'quirofano 2': 11.808361809738992, 'quirofano 3': 13.505506419924478, 'quirofano 4': 3.400724177084842, 'quirofano 5': 7.220484409540371, 'quirofano 6': 12.13695966261268, 'quirofano 7': 19.122553827532172, 'quirofano 8': 8.9645300027725}, {'quirofano 1': 9.971561789806854, 'quirofano 2': 6.750363787811527, 'quirofano 3': 7.507882728703784, 'quirofano 4': 16.399543085

In [ ]:
disponibilidad_camas = [CAMAS_TOTALES] * DIAS_SIMULACION

In [ ]:
'''TEST CAMAS DISPONIBLES'''

for dia in range(3):
    print(dia)
    # llegadas = generar_demanda_diaria()
    llegadas = 2
    print('el dia:', dia,'ingresaron:',llegadas, 'pacientes')
    if disponibilidad_camas[dia] >= llegadas:
        for i in range(llegadas):
            
            cant_dias_internacion = generar_dias_internacion()
            cant_dias_internacion = 1 if cant_dias_internacion < 1 else cant_dias_internacion
            dias_int = int(cant_dias_internacion)
            print('los pacientes se internaron:',cant_dias_internacion,'dias')
            print('numero redondeado:', dias_int)
            asignar_cama(dia, dias_int)
print(disponibilidad_camas)

In [ ]:
'''TEST QUIROFANOS'''
pacientes = []
for dia in range(10):
    paciente = paciente_base.copy()
    paciente['dia_internacion'] = dia
    paciente['dia_alta'] = generar_dias_internacion()
    pacientes.append(paciente)

# print(pacientes)
# Asignar quirofano
quirofanos_keys = list(quirofanos.keys())
for i, paciente in enumerate(pacientes):
    key = quirofanos_keys[i % len(quirofanos_keys)]
    quirofanos[key].append(paciente)

# print(quirofanos)

for key in quirofanos:
    if quirofanos[key]:
        paciente = quirofanos[key].pop(0)
        print(paciente['dia_alta'])


In [ ]:
import random
quirofanos_keys = list(quirofanos.keys())

pacientes = []
for dia in range(10):
    paciente = paciente_base.copy()
    paciente['dia_internacion'] = dia
    paciente['dia_alta'] = generar_dias_internacion()
    pacientes.append(paciente)

print(len(pacientes))

for paciente in pacientes:
    key = quirofanos_keys[random.randint(0,len(quirofanos_keys))-1]
    quirofanos[key].append(paciente)

print(quirofanos['quirofano 1'])
print(quirofanos['quirofano 2'])
print(quirofanos['quirofano 3'])
print(quirofanos['quirofano 4'])

In [ ]:
'''TEST ATENCION QUIROFANOS'''
import random

for dia in range(10):
    ocupacion_quirofanos_test = {
        'quirofano 1': 0,
        'quirofano 2': 0,
        'quirofano 3': 0,
        'quirofano 4': 0
    }

    dia_hoy = 0
    kits_disponibles_test = 10 
    pacientes = []
    for i in range(10):
        paciente = paciente_base.copy()
        paciente['dia_internacion'] = random.randint(0,i)
        paciente['dia_alta'] = generar_dias_internacion()
        pacientes.append(paciente)

    for paciente in pacientes:
        key = quirofanos_keys[random.randint(0,len(quirofanos_keys))-1]
        quirofanos[key].append(paciente)

    

    # while ( ((ocupacion_quirofanos_test['quirofano 1'] <= CANT_HORAS_ATENCION_QUIROFANO)
    #     or (ocupacion_quirofanos_test['quirofano 2'] <= CANT_HORAS_ATENCION_QUIROFANO)
    #     or (ocupacion_quirofanos_test['quirofano 3'] <= CANT_HORAS_ATENCION_QUIROFANO)
    #     or (ocupacion_quirofanos_test['quirofano 4'] <= CANT_HORAS_ATENCION_QUIROFANO))
    #         and hay_kits(kits_disponibles_test)):

    for key in quirofanos:
        print('la ocupacion de los quirofanos es:',ocupacion_quirofanos_test)    
        cirugias_diarias = generar_cantidad_cirugias_diarias()
        for cirugia in range(cirugias_diarias):
            if ocupacion_quirofanos_test[key] <= CANT_HORAS_ATENCION_QUIROFANO:
                if quirofanos[key]:
                    paciente = quirofanos[key].pop(0)
                    if dia_hoy <= paciente['dia_alta']:
                        if hay_kits(kits_disponibles_test):
                            ocupacion_quirofanos_test[key] += determinar_tiempo_operacion()
                            kits_disponibles_test -= 1
                            print(f'Operacion realizada en {key}. Tiempo de ocupacion ahora: {ocupacion_quirofanos_test[key]}')
                        else:
                            quirofanos[key].append(paciente)
                            print(f'Kits insuficientes. Paciente devuelto a {key}')
                    else:
                        print(f'Paciente en {key} no puede ser operado, dia alta {paciente["dia_alta"]} ha pasado.')
                else:
                    print(f'{key} no tiene pacientes en la cola.')

    print('tiempo total de ocupación del quirofano:', ocupacion_quirofanos_test)
    print('estado del quirofano:', quirofanos)
    print('Kits restantes:', kits_disponibles_test)